## 3. Parallelization

<img style="float: right;" src="../img/flex-logo.png" width="120"><br>

<div style="text-align: right"> <b>your name</b></div>
<div style="text-align: right"> Initial issue : 2025.10.02 </div>
<div style="text-align: right"> last update : 2025.10.02 </div>

개정 이력  
- `2025.10.02` : 노트북 

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
import asyncio
from typing import Optional
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import Runnable, RunnableParallel, RunnablePassthrough

In [3]:
try:
    llm: Optional[ChatOpenAI] = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
except Exception as e:
    print(f"언어 모델 초기화 오류: {e}")
    llm = None

### 0. 독립 체인 정의

In [4]:
summarize_chain: Runnable = (
    ChatPromptTemplate.from_messages([
        ("system", "다음 주제를 간결하게 요약하십시오:"),
        ("user", "{topic}")
    ])
    | llm
    | StrOutputParser()
)

questions_chain: Runnable = (
    ChatPromptTemplate.from_messages([
        ("system", "다음 주제에 대한 세 가지 흥미로운 질문을 생성하십시오:"),
        ("user", "{topic}")
    ])
    | llm
    | StrOutputParser()
)

terms_chain: Runnable = (
    ChatPromptTemplate.from_messages([
        ("system", "다음 주제에서 5-10개의 핵심 용어를 쉼표로 구분하여 식별하십시오:"),
        ("user", "{topic}")
    ])
    | llm
    | StrOutputParser()
)

### 1. 병렬 실행 블록 정의: 병렬 결과와 원래 주제를 다음 단계로 전달

In [5]:
map_chain = RunnableParallel(
    {
        "summary": summarize_chain,
        "questions": questions_chain,
        "key_terms": terms_chain,
        "topic": RunnablePassthrough(),  # 원래 주제를 그대로 전달
    }
)

### 2. 병렬 결과를 병합할 합성 프롬프트, 체인 정의

In [6]:
synthesis_prompt = ChatPromptTemplate.from_messages([
    ("system", """
    다음 정보를 기반으로 포괄적인 답변을 작성하세요.:
     요약: {summary}
     관련 질문: {questions}
     핵심 용어: {key_terms}
     """),
    ("user", "원래 주제: {topic}")
])

# 3. 병렬 결과를 합성 프롬프트로 직접 연결한 다음 LLM과 출력 파서로 전체 체인을 구성합니다
full_parallel_chain = map_chain | synthesis_prompt | llm | StrOutputParser()

### 3. 체인 실행

In [7]:
async def run_parallel_example(topic: str) -> None:
    """
    특정 주제로 병렬 처리 체인을 비동기적으로 호출하고 합성된 결과를 출력합니다.
    Args:
        topic: LangChain 체인으로 처리될 입력 주제
    """
    if not llm:
        print("LLM이 초기화되지 않았습니다. 예제를 실행할 수 없습니다.")
        return

    print(f"\n--- 주제 '{topic}'에 대한 병렬 LangChain 예제 실행 ---")

    try:
        # `ainvoke`에 대한 입력은 단일 'topic' 문자열이며,
        # `map_chain`의 각 실행 가능 객체로 전달됩니다
        response = await full_parallel_chain.ainvoke(topic)
        print("\n--- 최종 응답 ---")
        print(response)
    except Exception as e:
        print(f"\n체인 실행 중 오류 발생: {e}")

In [8]:
# Jupyter 노트북에서는 이미 이벤트 루프가 실행 중이므로 await를 직접 사용합니다
test_topic = "우주 탐사의 역사"
await run_parallel_example(test_topic)


--- 주제 '우주 탐사의 역사'에 대한 병렬 LangChain 예제 실행 ---

--- 최종 응답 ---
우주 탐사의 역사는 인류의 호기심과 과학적 진보의 상징으로, 20세기 중반부터 본격적으로 시작되었습니다. 이 시기는 인류가 지구를 넘어 우주를 탐험하려는 첫걸음을 내디딘 중요한 시점입니다. 

첫 번째로 주목할 만한 사건은 1957년 소련이 세계 최초의 인공위성인 스푸트니크 1호를 발사한 것입니다. 이는 우주 탐사의 시대를 여는 기폭제 역할을 하였고, 이후의 많은 우주 프로그램에 영감을 주었습니다. 스푸트니크의 발사는 국가 간의 우주 경쟁을 촉발시키고, 과학과 기술의 발전을 가속화하는 계기가 되었습니다. 이러한 경쟁은 나중에 미국의 아폴로 프로그램으로 이어져, 인류의 우주 탐사에서 중요한 이정표를 세우는 결과를 가져왔습니다.

1961년, 유리 가가린이 인류 최초로 유인 우주비행을 성공적으로 수행한 것은 또 다른 중요한 사건입니다. 이는 인간이 지구의 중력을 넘어 우주로 나아갈 수 있다는 가능성을 보여주었고, 이후의 유인 우주 탐사 미션에 대한 신뢰와 흥미를 높였습니다.

1969년 아폴로 11호의 달 착륙은 우주 탐사의 역사에서 상징적인 사건으로, 인류가 지구를 넘어 외계 천체에 발을 디딘 첫 번째 사례였습니다. 이 미션은 과학적 성취뿐 아니라, 인류의 단결과 탐험 정신을 보여주는 상징으로 남아 있습니다. 아폴로 11호의 성공은 이후의 우주 탐사 미션에 큰 영감을 주었으며, 다양한 국가와 민간 기업들이 우주 탐사에 도전하도록 자극하였습니다.

1990년대에 들어서면서 허블 우주 망원경의 발사는 우주 관측의 혁신을 가져왔습니다. 이 망원경은 깊은 우주를 관찰할 수 있는 능력을 통해 우주의 기원, 구조, 그리고 진화에 대한 이해를 크게 확장시켰습니다.

현재는 민간 우주 기업들이 적극적으로 참여하고 있으며, 특히 화성 탐사 및 유인 화성 미션 준비가 활발히 진행되고 있습니다. SpaceX와 같은 기업들은 재사용 가능한 로켓 기술을 개발하여 우주 여행의 